In [7]:
library(Seurat)
library(openxlsx)
library(readxl)
library(Matrix)
library(readr)

In [10]:
#Need to find out if all duplicates are nonzeros
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Fons")
#Upload matrices
data_fons_matrix <- readMM("data_fons_matrix.mtx")
#Upload original gene list
#data_fons_genes <- read.delim("data_fons_genes.tsv")
#data_fons_genes <- data_fons_genes$genes
data_fons_cellIDs <- read.delim("data_fons_cellIDs.tsv")
#rownames(data_fons_matrix) <- data_fons_genes
colnames(data_fons_matrix) <- data_fons_cellIDs$cellIDs

In [12]:
#Upload genes of interest (duplicates)
data_fons_genes_dup <- read.xlsx("data_fons_geneIDs_duplicates_forsum.xlsx")

In [13]:
#Find the zeros of these duplicates
#Initialize an empty list to store the sums
sums_list <- list()

#Iterate over each gene in your list
for (gene in data_fons_genes_dup$Old) {
  #Subset the count matrix for the current gene and calculate the sum
  gene_sum <- sum(data_fons_matrix[rownames(data_fons_matrix) == gene, ])
  
  #Add the sum to the sums_list with the gene name as the key
  sums_list[[gene]] <- gene_sum
}

In [11]:
#Upload manually edited geneIDs
data_fons_genes_manual <- read.xlsx("data_fons_geneIDs_int_removed_manualedit.xlsx")
rownames(data_fons_matrix) <- data_fons_genes_manual$New

In [12]:
#Check duplicates
#Find sums of duplicate genes (from manual edit)
duplicate_genes <- duplicated(rownames(data_fons_matrix))
cat("Duplicate gene IDs:\n")
#print(rownames(data_fons_matrix)[duplicate_genes])
duplicated <- rownames(data_fons_matrix)[duplicate_genes]
length(duplicated)
#matches excel!

Duplicate gene IDs:


[1] 172

In [13]:
#Filter zeros
row_sums <- rowSums(data_fons_matrix)
names_with_zeros <- names(row_sums[row_sums == 0])
length(names_with_zeros)

[1] 30

In [14]:
filtered_data_fons_matrix <- data_fons_matrix[row_sums != 0, ]

In [15]:
#Concatenating replicates
filtered_data_fons_matrix <- as(filtered_data_fons_matrix, "dgCMatrix")

'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").



In [16]:
gene_names <- rownames(filtered_data_fons_matrix)
unique_gene_names <- unique(gene_names)
gene_factors <- factor(gene_names, levels = unique_gene_names)

In [17]:
aggregated_counts <- rowsum(as.matrix(filtered_data_fons_matrix), group = gene_factors)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 73.1 GiB”


In [27]:
#Creating a seurat object
data_fons_metadata <- read_csv("data_fons_metadata_final.csv")

New names:
• `` -> `...1`
Rows: 342668 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): ...1, orig.ident, patientID, study, cells_or_nuclei, tissue, condit...
dbl (6): nCount_RNA, nFeature_RNA, percent.mt, age, RNA_snn_res.0.8, seurat_...
lgl (2): sex, smoking

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [28]:
data_fons_metadata <- as.data.frame(data_fons_metadata)

In [29]:
rownames(data_fons_metadata) <- data_fons_metadata$`...1`

In [30]:
data_fons_metadata$`...1` <- NULL
data_fons_metadata$`...2` <- NULL

In [32]:
#Create Seurat Object
data_fons_seurat <- CreateSeuratObject(counts = aggregated_counts, meta.data = data_fons_metadata)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [34]:
unique(data_fons_seurat$cells)

[1] "T cells"            "B cells"            "Fibroblasts"       
 [4] "Macrophages"        "NK cells"           "Monocytes"         
 [7] "Endothelial cells"  "SMCs"               "Plasma cells"      
[10] "Mitotic"            "Epithelial cells"   "Ciliated cells"    
[13] "Mast cells"         "Lymph. endo. cells" "pDCs"

In [35]:
save(data_fons_seurat, file = "data_fons_seurat_final.RData")